# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv("WeatherPy.csv")
cities.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,Ambikāpur,23.12,83.20,79.84,71,35,2.80,IN
1,Hermanus,-34.42,19.23,57.99,73,46,5.88,ZA
2,Shima,24.45,117.81,82.40,88,40,4.47,CN
3,Chālūs,36.66,51.42,69.80,77,0,4.70,IR
4,Kapaa,22.08,-159.32,84.20,66,20,10.29,US


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
humidity = cities["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = cities[["Latitude","Longitude"]]

In [5]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False,max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
hotel_df = cities.loc[(cities["Max Temp"] > 77) & (cities["Max Temp"] <88) &(cities["Cloudiness"]==0), :]
hotel_df = hotel_df.dropna(how="any")
hotel_df.reset_index(inplace = True)
del hotel_df["index"]
hotel_df.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,Tarata,-17.62,-66.02,78.80,9,0,3.36,BO
1,Nouakchott,18.09,-15.98,78.80,78,0,12.75,MR
2,Agadez,19.75,10.25,83.23,15,0,3.62,NE
3,Cabra,37.47,-4.44,82.00,53,0,3.94,ES
4,Gogoşari,43.87,25.70,78.01,58,0,1.01,RO


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
hotellist = []

for x in range(len(hotel_df)):
    lat = hotel_df.loc[x]['Latitude']
    lng = hotel_df.loc[x]['Longitude']
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types": "hotel",
        "key": g_key,
        }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    request=requests.get(base_url, params=params)
    json = request.json()
    try:
        hotellist.append(json["results"][0]["name"])
    except:
        hotellist.append(" ")
hotel_df["Hotel Name"] = hotellist
hotel_df = hotel_df.dropna(how='any')
hotel_df.head()


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
0,Tarata,-17.62,-66.02,78.80,9,0,3.36,BO,Tarata
1,Nouakchott,18.09,-15.98,78.80,78,0,12.75,MR,Nouakchott
2,Agadez,19.75,10.25,83.23,15,0,3.62,NE,
3,Cabra,37.47,-4.44,82.00,53,0,3.94,ES,Cabra
4,Gogoşari,43.87,25.70,78.01,58,0,1.01,RO,Putineiu


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))